In [ ]:
!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip ml-latest-small.zip

--2020-09-25 18:43:42--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  1.45MB/s    in 0.6s    

2020-09-25 18:43:43 (1.45 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import Normalizer , scale
from sklearn.model_selection import GridSearchCV , KFold , cross_val_score
from sklearn.metrics import mean_squared_log_error,mean_squared_error, r2_score,mean_absolute_error 
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
%matplotlib inline

In [ ]:
import pandas as pd
import re

movies = pd.read_csv('ml-latest-small/movies.csv')
movies.head()

movieId  ...                                       genres
0        1  ...  Adventure|Animation|Children|Comedy|Fantasy
1        2  ...                   Adventure|Children|Fantasy
2        3  ...                               Comedy|Romance
3        4  ...                         Comedy|Drama|Romance
4        5  ...                                       Comedy

[5 rows x 3 columns]

In [ ]:
ratings = pd.read_csv('ml-latest-small/ratings.csv').sort_values(by='movieId')
ratings['movieId'] = ratings['movieId'].replace({v: k for k, v in dict(movies['movieId']).items()})

In [ ]:
movies['Movie Name'] = movies['title'].apply(lambda x: x[:-6])
movies['Year'] = movies['title'].apply(lambda x: re.sub('[^0-9]', ' ', x))
def split_series(ser,sep):
    return pd.Series(ser.str.cat(sep=sep).split(sep=sep)) 

movies_proc=(movies.groupby(df.columns.drop('genres').tolist()) #group by all but one column
          ['genres'] #select the column to be split
          .apply(split_series,sep='|') # split genres in each group
         .reset_index(drop=True,level=-1).reset_index()) #remove extra index created

movies_proc.head()

movieId             title              Name              Year     genres
0        1  Toy Story (1995)  Toy Story                    1995   Adventure
1        1  Toy Story (1995)  Toy Story                    1995   Animation
2        1  Toy Story (1995)  Toy Story                    1995    Children
3        1  Toy Story (1995)  Toy Story                    1995      Comedy
4        1  Toy Story (1995)  Toy Story                    1995     Fantasy

In [ ]:
movie_data = pd.merge(movies_proc, ratings, on='movieId')

movie_data_dummy= pd.get_dummies(movie_data['genres'])
movie_data = pd.concat([movie_data, movie_data_dummy], axis=1)
movie_data = movie_data.drop(['title','timestamp','genres','Name','Year'], axis = 1)

movie_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148725 entries, 0 to 148724
Data columns (total 22 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   movieId      148725 non-null  int64  
 1   userId       148725 non-null  int64  
 2   rating       148725 non-null  float64
 3   Action       148725 non-null  uint8  
 4   Adventure    148725 non-null  uint8  
 5   Animation    148725 non-null  uint8  
 6   Children     148725 non-null  uint8  
 7   Comedy       148725 non-null  uint8  
 8   Crime        148725 non-null  uint8  
 9   Documentary  148725 non-null  uint8  
 10  Drama        148725 non-null  uint8  
 11  Fantasy      148725 non-null  uint8  
 12  Film-Noir    148725 non-null  uint8  
 13  Horror       148725 non-null  uint8  
 14  IMAX         148725 non-null  uint8  
 15  Musical      148725 non-null  uint8  
 16  Mystery      148725 non-null  uint8  
 17  Romance      148725 non-null  uint8  
 18  Sci-Fi       148725 non-

In [ ]:
train, test = train_test_split(movie_data, test_size=0.1)
ROW_COUNT = train.shape[0]

train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 133852 entries, 7539 to 130169
Data columns (total 22 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   movieId      133852 non-null  int64  
 1   userId       133852 non-null  int64  
 2   rating       133852 non-null  float64
 3   Action       133852 non-null  uint8  
 4   Adventure    133852 non-null  uint8  
 5   Animation    133852 non-null  uint8  
 6   Children     133852 non-null  uint8  
 7   Comedy       133852 non-null  uint8  
 8   Crime        133852 non-null  uint8  
 9   Documentary  133852 non-null  uint8  
 10  Drama        133852 non-null  uint8  
 11  Fantasy      133852 non-null  uint8  
 12  Film-Noir    133852 non-null  uint8  
 13  Horror       133852 non-null  uint8  
 14  IMAX         133852 non-null  uint8  
 15  Musical      133852 non-null  uint8  
 16  Mystery      133852 non-null  uint8  
 17  Romance      133852 non-null  uint8  
 18  Sci-Fi       133852 n

In [ ]:
num_movies= movie_data.movieId.nunique()
num_users = movie_data.userId.nunique()
movie_data.shape[0]

148725

In [ ]:
UNIQUE_MOVIE_IDS = movie_data['movieId'].unique()

In [ ]:
def EmbeddingRec(EMBEDDING_SIZE, NUM_MOVIES, NUM_USERS, ROW_COUNT):
    movie_input = keras.Input(shape=(1,), name='movieId')

    movie_emb = layers.Embedding(output_dim=EMBEDDING_SIZE, input_dim=NUM_MOVIES, input_length=ROW_COUNT, name='movie_emb')(movie_input)
    movie_vec = layers.Flatten(name='FlattenMovie')(movie_emb)

    movie_model = keras.Model(inputs=movie_input, outputs=movie_vec)
    
    user_input = keras.Input(shape=(1,), name='userId')

    user_emb = layers.Embedding(output_dim=EMBEDDING_SIZE, input_dim=NUM_USERS, input_length=ROW_COUNT, name='user_emb')(user_input)
    user_vec = layers.Flatten(name='FlattenUser')(user_emb)

    user_model = keras.Model(inputs=user_input, outputs=user_vec)
    
    merged = layers.Dot(name = 'dot_product', normalize = True, axes = 2)([movie_emb, user_emb])
    merged_dropout = layers.Dropout(0.2)(merged)
    
    
    dense_1 = layers.Dense(70,name='FullyConnected-1')(merged)
    dropout_1 = layers.Dropout(0.2,name='Dropout_1')(dense_1)

    dense_2 = layers.Dense(50,name='FullyConnected-2')(dropout_1)
    dropout_2 = layers.Dropout(0.2,name='Dropout_2')(dense_2)

    dense_3 = keras.layers.Dense(20,name='FullyConnected-3')(dropout_2)
    dropout_3 = keras.layers.Dropout(0.2,name='Dropout_3')(dense_3)

    dense_4 = keras.layers.Dense(10,name='FullyConnected-4', activation='relu')(dropout_3)

    result = layers.Dense(1, name='result', activation="relu") (dense_4)

    adam = keras.optimizers.Adam(lr=0.001)
    model = keras.Model([movie_input, user_input], result)
    model.compile(optimizer=adam,loss= 'mean_absolute_error')
    return model, movie_model, user_model

In [ ]:
EMBEDDING_SIZE = 10

model, movie_model, user_model = EmbeddingRec(EMBEDDING_SIZE, num_movies,num_users, ROW_COUNT)

In [ ]:

model.summary()

Model: "functional_23"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
movieId (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
userId (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
movie_emb (Embedding)           (None, 1, 10)        53910       movieId[0][0]                    
__________________________________________________________________________________________________
user_emb (Embedding)            (None, 1, 10)        6100        userId[0][0]                     
______________________________________________________________________________________

In [ ]:
callbacks = [keras.callbacks.EarlyStopping('val_loss', patience=10),keras.callbacks.ModelCheckpoint('besttest.h5', save_best_only=True)]

history = model.fit([train.movieId, train.userId],train.rating, batch_size=100, epochs =50, validation_data = ([test.movieId, test.userId],test.rating), verbose = 1, callbacks = callbacks)

Epoch 1/50


InvalidArgumentError: ignored

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
import matplotlib.pyplot as plt
plt.plot(history.history['loss'] , 'g')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(['train'], loc='upper right')
plt.grid(True)
plt.show()

NameError: ignored